In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPool2D, Dropout from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from PIL import Image
from tensorflow.keras.optimizers import Adam
%matplotlib inline
data_dir = '../input/gtsrb-german-traffic-sign'
train_path = '../input/gtsrb-german-traffic-sign/Train'
test_path = '../input/gtsrb-german-traffic-sign/'
IMG_HEIGHT = 30
IMG_WIDTH = 30
channels = 3
NUM_CATEGORIES = len(os.listdir(train_path))
NUM_CATEGORIES
classes = { 0:'Speed limit (20km/h)',
1:'Speed limit (30km/h)', 2:'Speed limit (50km/h)', 3:'Speed limit (60km/h)', 4:'Speed limit (70km/h)', 5:'Speed limit (80km/h)', 6:'End of speed limit (80km/h)', 7:'Speed limit (100km/h)', 8:'Speed limit (120km/h)', 9:'No passing',
10:'No passing veh over 3.5 tons', 11:'Right-of-way at intersection', 12:'Priority road',
13:'Yield',
14:'Stop',
15:'No vehicles',
16:'Veh > 3.5 tons prohibited', 17:'No entry',
18:'General caution', 19:'Dangerous curve left', 20:'Dangerous curve right', 21:'Double curve',
22:'Bumpy road',
23:'Slippery road',
24:'Road narrows on the right', 25:'Road work',
26:'Traffic signals',
27:'Pedestrians',
28:'Children crossing',
29:'Bicycles crossing',
30:'Beware of ice/snow',
31:'Wild animals crossing',
32:'End speed + passing limits', 33:'Turn right ahead',
34:'Turn left ahead',
35:'Ahead only',
36:'Go straight or right',
37:'Go straight or left',
38:'Keep right',
39:'Keep left',
40:'Roundabout mandatory', 41:'End of no passing',
42:'End no passing veh > 3.5 tons' }
import random
from matplotlib.image import imread test = pd.read_csv(data_dir + '/Test.csv') imgs = test["Path"].values plt.figure(figsize=(25,25))
for i in range(1,26):
plt.subplot(5,5,i)
random_img_path = data_dir + '/' + random.choice(imgs) rand_img = imread(random_img_path) plt.imshow(rand_img)
plt.grid(visible=0)
plt.xlabel(rand_img.shape[1], fontsize = 20)#width of image
plt.ylabel(rand_img.shape[0], fontsize = 20)#height of image def load_data(data_dir):
images = list() labels = list()
for category in range(NUM_CATEGORIES): categories = os.path.join(data_dir, str(category)) for img in os.listdir(categories):
img = load_img(os.path.join(categories, img), target_size=(30, 30))
image = img_to_array(img) images.append(image)
labels.append(category)
return images, labels
images, labels = load_data(train_path)
labels = to_categorical(labels)
x_train, x_test, y_train, y_test = train_test_split(
np.array(images),
labels,
test_size=0.3 , random_state=42, shuffle=True
)


In [ ]:
x_train = x_train/255
x_test = x_test/255
print("X_train.shape", x_train.shape)
print("X_valid.shape", x_test.shape)
print("y_train.shape", y_train.shape)
print("y_valid.shape", y_test.shape)
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(3,3), activation="relu", input_shape=(IMG_HEIGHT,IMG_WIDTH,3))) model.add(Conv2D(filters=32, kernel_size=(3,3), activation="relu")) model.add(MaxPool2D(pool_size=(2,2))) model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation="relu")) model.add(Conv2D(filters=128, kernel_size=(3,3), activation="relu")) model.add(MaxPool2D(pool_size=(2,2))) model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(512, activation="relu")) model.add(Dropout(rate=0.5))
model.add(Dense(43, activation="softmax"))
model.summary()
model.compile(
loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']
)
aug = ImageDataGenerator(
rotation_range=10, zoom_range=0.15, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.15, horizontal_flip=False, vertical_flip=False, fill_mode="nearest")
EPOCHS = 30
history = model.fit(aug.flow(x_train, y_train, batch_size = 32),
validation_data = (x_test, y_test),
epochs=EPOCHS )
loss, accuracy = model.evaluate(x_test, y_test)
print('test set accuracy: ', accuracy * 100) plt.figure(figsize = (16, 16))
start_index = 0
for i in range(25):
plt.subplot(5, 5, i + 1)
plt.grid(False)
plt.xticks([])
plt.yticks([])
prediction = pred[start_index + i] actual = test_labels[start_index + i] col = 'g'
if prediction != actual: col = 'r'
plt.xlabel('Actual={} || Pred={}'.format(actual, prediction), color = col) plt.imshow(X_test[start_index + i])
plt.show()